In [1]:
import pandas as pd 
import numpy as np 

In [2]:
#read merged data
merged_data = pd.read_csv('data/clean/final_merged_data.csv', na_values=[""])
merged_data

,year-quarter,state,sex,age_group,total_bene_th_elig,total_bene_telehealth,pct_telehealth,service_type,servicecount,rateper1000beneficiaries,group,subgroup,value
0,2018-1,Alabama,NaN,NaN,NaN,NaN,NaN,Live audio/video,"6,905",5.6,NaN,NaN,NaN
1,2018-1,Alabama,NaN,NaN,NaN,NaN,NaN,Live audio/video,"4,821",3.9,NaN,NaN,NaN
2,2018-1,Alabama,NaN,NaN,NaN,NaN,NaN,Live audio/video,"2,269",1.8,NaN,NaN,NaN
3,2018-2,Alabama,NaN,NaN,NaN,NaN,NaN,Live audio/video,"2,290",1.8,NaN,NaN,NaN
4,2018-2,Alabama,NaN,NaN,NaN,NaN,NaN,Live audio/video,"2,380",1.9,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
199040,2024-2,Wyoming,NaN,65-74,32849.0,2025.0,0.0616,NaN,NaN,NaN,NaN,NaN,NaN
199041,2024-2,Wyoming,NaN,75-84,21882.0,1234.0,0.0564,NaN,NaN,NaN,NaN,NaN,NaN
199042,2024-2,Wyoming,NaN,85 and over,7560.0,387.0,0.0512,NaN,NaN,NaN,NaN,NaN,NaN
199043,2024-2,Wyoming,NaN,NaN,46806.0,3411.0,0.0729,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
telemedicine_services = pd.read_csv('data/raw/Telehealth Services Provided to the MedicaidCHIP Population.csv')
telemedicine_services

,State,Year,Month,TelehealthType,ServiceType,ServiceCount,RatePer1000Beneficiaries,DataQuality
0,Alabama,2018,201801,Live audio/video,All,"6,905",5.6,NaN
1,Alabama,2018,201801,Other telehealth,All,-,0,NaN
2,Alabama,2018,201801,Remote patient monitoring,All,-,0,NaN
3,Alabama,2018,201801,Store and forward,All,-,0,NaN
4,Alabama,2018,201802,Live audio/video,All,"4,821",3.9,NaN
...,...,...,...,...,...,...,...,...
12715,Wyoming,2022,202211,Store and forward,All,DS,DS,NaN
12716,Wyoming,2022,202212,Live audio/video,All,"3,189",35.7,NaN
12717,Wyoming,2022,202212,Other telehealth,All,DS,DS,NaN
12718,Wyoming,2022,202212,Remote patient monitoring,All,33,0,NaN


In [4]:
# Data Cleaning for Telehealth Services Provided to the Medicaid and CHIP Population 

# Drop irrelevant columns 
telemedicine_services_clean = telemedicine_services.drop(columns = ['ServiceType']) 

# Remove rows with "Unusable" in DataQuality column
telemedicine_services_clean = telemedicine_services_clean[telemedicine_services_clean['DataQuality'] != 'Unusable']

# Remove rows with "DS" in any column
telemedicine_services_clean = telemedicine_services_clean[~telemedicine_services_clean.eq('DS').any(axis=1)]
telemedicine_services_clean = telemedicine_services_clean.drop(columns = ['DataQuality'])

# Remove rows with missing values in ServiceCount and RatePer1000Beneficiaries
telemedicine_services_clean = telemedicine_services_clean[
    (telemedicine_services_clean['ServiceCount'] != '-') & 
    (telemedicine_services_clean['RatePer1000Beneficiaries'] != '0')
]

telemedicine_services_clean['Month'] = telemedicine_services_clean['Month']%100


In [5]:
telemedicine_services_clean

,State,Year,Month,TelehealthType,ServiceCount,RatePer1000Beneficiaries
0,Alabama,2018,1,Live audio/video,"6,905",5.6
4,Alabama,2018,2,Live audio/video,"4,821",3.9
8,Alabama,2018,3,Live audio/video,"2,269",1.8
12,Alabama,2018,4,Live audio/video,"2,290",1.8
16,Alabama,2018,5,Live audio/video,"2,380",1.9
...,...,...,...,...,...,...
12704,Wyoming,2022,9,Live audio/video,"2,977",34.2
12708,Wyoming,2022,10,Live audio/video,"2,987",34.2
12710,Wyoming,2022,10,Remote patient monitoring,40,0.5
12712,Wyoming,2022,11,Live audio/video,"3,247",36.7


In [6]:
# Export telemedicine_services_clean dataset into csv file

telemedicine_services_clean.to_csv('data/clean/tele_service_for_sql.csv', index=False)